In [1]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as bs

# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}

In [ ]:
leagueTables = [
    "https://www.transfermarkt.com/premier-league/tabelle/wettbewerb/GB1/saison_id/2024",
    "https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1/saison_id/2024",
    "https://www.transfermarkt.com/ligue-1/tabelle/wettbewerb/FR1/saison_id/2024",
    "https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1/saison_id/2024",
    "https://www.transfermarkt.com/serie-a/tabelle/wettbewerb/IT1/saison_id/2024",
]

df0 = pd.DataFrame()
for link in leagueTables:
    re = req.get(link, headers=headers)
    so = bs(re.content, "html.parser")
    tb = so.select_one('table:contains("Goals")')
    df1 = pd.DataFrame(
        {
            "Mini Team Crest": [
                (i["src"]) for i in tb.select("tbody td:nth-of-type(2) a img")
            ],
            "Team": [(i["title"]) for i in tb.select("tbody td:nth-of-type(2) a img")],
            "Team Link": [
                ("https://www.transfermarkt.com" + i["href"] + "/plus/1")
                for i in tb.select("tbody td:nth-of-type(3) a:nth-of-type(1)")
            ],
            "MP": [(i.text) for i in tb.select("tbody td:nth-of-type(4)")],
            "W": [(i.text) for i in tb.select("tbody td:nth-of-type(5)")],
            "D": [(i.text) for i in tb.select("tbody td:nth-of-type(6)")],
            "L": [(i.text) for i in tb.select("tbody td:nth-of-type(7)")],
            "Goals": [(i.text) for i in tb.select("tbody td:nth-of-type(8)")],
            "+/-": [(i.text) for i in tb.select("tbody td:nth-of-type(9)")],
            "Pts": [(i.text) for i in tb.select("tbody td:nth-of-type(10)")],
        }
    )

    def decideLeague(x):
        if link == leagueTables[0]:
            x = "EPL"
            return x
        elif link == leagueTables[1]:
            x = "La Liga"
            return x
        elif link == leagueTables[2]:
            x = "Ligue 1"
            return x
        elif link == leagueTables[3]:
            x = "Bundesliga"
            return x
        elif link == leagueTables[4]:
            x = "Serie A"
            return x

    df1["League"] = df1.apply(decideLeague, axis=1)

    df0 = pd.concat([df0, df1], axis=0)


def clubCrestBig(x):
    return x["Mini Team Crest"].replace("tiny", "head")


df0["Big Team Crest"] = df0.apply(clubCrestBig, axis=1)
df0.insert(1, "Big Team Crest", df0.pop("Big Team Crest"))


def teamLink(x):
    return x["Team Link"].replace("/spielplan/", "/kader/")


df0["Team Link"] = df0.apply(teamLink, axis=1)


def teamLinkSecondFirst(x):
    return x["Team Link"].replace("/kader/", "/leistungsdaten/")


df0["Team Link 2"] = df0.apply(teamLinkSecondFirst, axis=1)


def teamLinkSecondSecond(x):
    if x["League"] == "EPL":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/GB1%262024/")
    elif x["League"] == "La Liga":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/ES1%262024/")
    elif x["League"] == "Ligue 1":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/FR1%262024/")
    elif x["League"] == "Bundesliga":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/L1%262024/")
    elif x["League"] == "Serie A":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/IT1%262024/")


df0["Team Link 2"] = df0.apply(teamLinkSecondSecond, axis=1)


df0["Pts/MP"] = df0["Pts"].astype(int) / df0["MP"].astype(int)
df0["Pts/MP"] = df0["Pts/MP"].round(2)
df2 = df0.sort_values(by="Team", ascending=True)
df3 = df2.reset_index(drop=True)

df3

In [ ]:
listPlayer = pd.DataFrame()
for l in df3["Team Link"]:
    r = req.get(
        l,
        headers=headers,
    )
    s = bs(r.content, "html.parser")
    t = s.find("table", class_="items")
    rows = s.select("tbody tr")
    
    innerListPlayer = pd.DataFrame(
        {
            "Player": [(i["title"]) for i in t.select("tbody td:nth-of-type(1) img")],
            "Position": [
                (i.text)
                for i in t.select("tbody td:nth-of-type(2) tr td:nth-of-type(1)")
            ][1::2],
            "Age": [(i.text) for i in t.select("tbody td:nth-of-type(3)")],
            "Height": [(i.text) for i in t.select("tbody td:nth-of-type(5)")],
            "Foot": [(i.text) for i in t.select("tbody td:nth-of-type(6)")],
            "Market Value": [(i.text) for i in t.select("tbody .rechts.hauptlink")],
        }
    )

    dataNation = [[td["title"].strip() for td in row.select("td:nth-of-type(4) img")] for row in rows]
    filteredDataNation = [sublist for sublist in dataNation if sublist]
    dataNationFlag = [[td["src"].strip() for td in row.select("td:nth-of-type(4) img")] for row in rows]
    filteredDataNationFlag = [sublist for sublist in dataNationFlag if sublist]

    filteredDataNationList = []
    for i in filteredDataNation:
        filteredDataNationList.append(i[0])

    filteredDataNationFlagList = []
    for i in filteredDataNationFlag:
        filteredDataNationFlagList.append(i[0])

    innerListPlayer["Nation"] = filteredDataNationList
    innerListPlayer["Nation Flag Link"] = filteredDataNationFlagList

    teams = s.find_all(
        class_="data-header__headline-wrapper data-header__headline-wrapper--oswald"
    )
    teamName = [team.get_text(strip=True) for team in teams]

    def addTeamName(c):
        c = teamName[0]
        return c

    innerListPlayer["Team"] = innerListPlayer.apply(addTeamName, axis=1)

    listPlayer = pd.concat([listPlayer, innerListPlayer], axis=0)

listPlayer["Age"] = listPlayer["Age"].str[-3:-1]
listPlayer["Position"] = listPlayer["Position"].replace("\n", "", regex=True)
listPlayer["Position"] = listPlayer["Position"].str.strip()
listPlayer.index = range(0, len(listPlayer))

listPlayer

In [ ]:
marketValueOne = listPlayer[listPlayer["Market Value"].str.contains("k")]
listPlayer.drop(
    listPlayer[listPlayer["Market Value"].str.contains("k")].index,
    inplace=True,
)

marketValueSecond = marketValueOne[
    marketValueOne["Market Value"].str.contains("\xa0\xa0")
]
marketValueOne.drop(
    marketValueOne[marketValueOne["Market Value"].str.contains("\xa0\xa0")].index,
    inplace=True,
)

In [ ]:
marketValueSeventh = pd.DataFrame()
marketValueSeventh["Market Value"] = marketValueOne["Market Value"].str[1:-1]
marketValueSeventh["Market Value"] = pd.to_numeric(marketValueSeventh["Market Value"])

marketValueSeventh

In [ ]:
valueList = []
for value in marketValueSeventh["Market Value"]:
    if value < 100 and value > 0:
        value = 0.1
        valueList.append(value)
    else:
        value = round((value / 1000), 1)
        valueList.append(value)

valueList

In [ ]:
marketValueSeventh["Market Value"] = valueList
marketValueSeventh

In [ ]:
marketValueEighth = pd.DataFrame()
marketValueEighth["Market Value"] = marketValueSecond["Market Value"].str[1:-3]
marketValueEighth["Market Value"] = pd.to_numeric(marketValueEighth["Market Value"])

marketValueEighth

In [ ]:
otherValueList = []
for v in marketValueEighth["Market Value"]:
    if v < 100 and v > 0:
        v = 0.1
        otherValueList.append(v)
    else:
        v = round((v / 1000), 1)
        otherValueList.append(v)

otherValueList

In [ ]:
marketValueEighth["Market Value"] = otherValueList
marketValueEighth

In [ ]:
marketValueOne.drop(columns=["Market Value"], inplace=True)
marketValueSecond.drop(columns=["Market Value"], inplace=True)

In [12]:
marketValueThird = pd.concat([marketValueOne, marketValueSeventh], axis=1)
marketValueNinth = pd.concat([marketValueSecond, marketValueEighth], axis=1)

marketValueTenth = pd.concat([marketValueThird, marketValueNinth], axis=0)

In [ ]:
marketValueTenth

In [ ]:
marketValueFourth = listPlayer[listPlayer["Market Value"].str.contains("m")]
listPlayer.drop(
    listPlayer[listPlayer["Market Value"].str.contains("m")].index,
    inplace=True,
)

marketValueFifth = marketValueFourth[marketValueFourth["Market Value"].str.contains("\xa0\xa0")]
marketValueFourth.drop(
    marketValueFourth[marketValueFourth["Market Value"].str.contains("\xa0\xa0")].index,
    inplace=True,
)

In [15]:
listPlayer["Market Value"] = 0.0
listPlayer.insert(4, "Market Value", listPlayer.pop("Market Value"))

In [ ]:
listPlayer

In [ ]:
marketValueSixth = pd.DataFrame()
marketValueSixth["Market Value"] = marketValueFourth["Market Value"].str[1:-1]
marketValueSixth["Market Value"] = pd.to_numeric(marketValueSixth["Market Value"])

marketValueSixth

In [ ]:
marketValueEleventh = pd.DataFrame()
marketValueEleventh["Market Value"] = marketValueFifth["Market Value"].str[1:-3]
marketValueEleventh["Market Value"] = pd.to_numeric(marketValueEleventh["Market Value"])

marketValueEleventh

In [ ]:
marketValueFourth.drop(columns=["Market Value"], inplace=True)
marketValueFifth.drop(columns=["Market Value"], inplace=True)

In [20]:
marketValueTwelfth = pd.concat([marketValueFourth, marketValueSixth], axis=1)
marketValueThirteenth = pd.concat([marketValueFifth, marketValueEleventh], axis=1)

marketValueFourteenth = pd.concat([marketValueTwelfth, marketValueThirteenth], axis=0)

In [ ]:
marketValueFourteenth

In [ ]:
finalListPlayer = pd.concat(
    [marketValueTenth, marketValueFourteenth, listPlayer], axis=0
)
finalListPlayer = finalListPlayer.reset_index(drop=True)
finalListPlayer

In [ ]:
listPlayerDetail = pd.DataFrame()
for li in df3["Team Link 2"]:

    res = req.get(
        li,
        headers=headers,
    )
    sou = bs(res.content, "html.parser")
    tbl = sou.select_one('table:contains("PPG")')
    innerListPlayerDetail = pd.DataFrame(
        {
            "Player": [(i["title"]) for i in tbl.select("tbody td:nth-of-type(1) img")],
            "Position": [
                (i.text)
                for i in tbl.select("tbody td:nth-of-type(2) tr td:nth-of-type(1)")
            ][1::2],
            "Age": [(i.text) for i in tbl.select("tbody td:nth-of-type(3)")],
            "MP": [(i.text) for i in tbl.select("tbody td:nth-of-type(6)")],
            "Gls": [(i.text) for i in tbl.select("tbody td:nth-of-type(7)")],
            "Asts": [(i.text) for i in tbl.select("tbody td:nth-of-type(8)")],
            "YC": [(i.text) for i in tbl.select("tbody td:nth-of-type(9)")],
            "Second YC": [(i.text) for i in tbl.select("tbody td:nth-of-type(10)")],
            "RC": [(i.text) for i in tbl.select("tbody td:nth-of-type(11)")],
            "Subs On": [(i.text) for i in tbl.select("tbody td:nth-of-type(12)")],
            "Min": [(i.text) for i in tbl.select("tbody td:nth-of-type(15)")],
        }
    )

    teams = sou.find_all(
        class_="data-header__headline-wrapper data-header__headline-wrapper--oswald"
    )
    teamNameDetail = [team.get_text(strip=True) for team in teams]

    def addTeamNameDetail(c):
        c = teamNameDetail[0]
        return c

    innerListPlayerDetail["Team"] = innerListPlayerDetail.apply(
        addTeamNameDetail, axis=1
    )

    listPlayerDetail = pd.concat([listPlayerDetail, innerListPlayerDetail], axis=0)

In [ ]:
listPlayerDetail = listPlayerDetail.reset_index(drop=True)
listPlayerDetail

In [25]:
finalListPlayer["MP"] = ""
finalListPlayer["Gls"] = ""
finalListPlayer["Asts"] = ""
finalListPlayer["YC"] = ""
finalListPlayer["Second YC"] = ""
finalListPlayer["RC"] = ""
finalListPlayer["Subs On"] = ""
finalListPlayer["Min"] = ""
finalListPlayer["St"] = ""

for n in range(len(finalListPlayer["Player"])):
    for m in range(len(listPlayerDetail["Player"])):
        if ((finalListPlayer.loc[n, "Player"] == listPlayerDetail.loc[m, "Player"]) and (finalListPlayer.loc[n, "Position"] == listPlayerDetail.loc[m, "Position"]) and (finalListPlayer.loc[n, "Age"] == listPlayerDetail.loc[m, "Age"]) and (finalListPlayer.loc[n, "Team"] == listPlayerDetail.loc[m, "Team"])):
            finalListPlayer.loc[n, "MP"] = listPlayerDetail.loc[m, "MP"]
            finalListPlayer.loc[n, "Gls"] = listPlayerDetail.loc[m, "Gls"]
            finalListPlayer.loc[n, "Asts"] = listPlayerDetail.loc[m, "Asts"]
            finalListPlayer.loc[n, "YC"] = listPlayerDetail.loc[m, "YC"]
            finalListPlayer.loc[n, "Second YC"] = listPlayerDetail.loc[m, "Second YC"]
            finalListPlayer.loc[n, "RC"] = listPlayerDetail.loc[m, "RC"]
            finalListPlayer.loc[n, "Subs On"] = listPlayerDetail.loc[m, "Subs On"]
            finalListPlayer.loc[n, "Min"] = listPlayerDetail.loc[m, "Min"]

In [ ]:
uniqueValues = finalListPlayer["MP"].unique()
uniqueValues

In [ ]:
finalListPlayerFoot = finalListPlayer[finalListPlayer["Foot"].str.contains("\xa0")]
finalListPlayer.drop(
    finalListPlayer[finalListPlayer["Foot"].str.contains("\xa0")].index,
    inplace=True,
)
finalListPlayerFoot["Foot"] = finalListPlayerFoot["Foot"].str[:-10]

finalListPlayer = pd.concat([finalListPlayer, finalListPlayerFoot], axis=0)

In [ ]:
finalListPlayer["MP"] = finalListPlayer["MP"].replace("Not used during this season", 0)
finalListPlayer["MP"] = finalListPlayer["MP"].replace("Not in squad during this season", 0)
finalListPlayer["MP"] = finalListPlayer["MP"].replace("", 0)
finalListPlayer["Gls"] = finalListPlayer["Gls"].replace("-", 0)
finalListPlayer["Gls"] = finalListPlayer["Gls"].replace("", 0)
finalListPlayer["Asts"] = finalListPlayer["Asts"].replace("-", 0)
finalListPlayer["Asts"] = finalListPlayer["Asts"].replace("", 0)
finalListPlayer["YC"] = finalListPlayer["YC"].replace("-", 0)
finalListPlayer["YC"] = finalListPlayer["YC"].replace("", 0)
finalListPlayer["Second YC"] = finalListPlayer["Second YC"].replace("-", 0)
finalListPlayer["Second YC"] = finalListPlayer["Second YC"].replace("", 0)
finalListPlayer["RC"] = finalListPlayer["RC"].replace("-", 0)
finalListPlayer["RC"] = finalListPlayer["RC"].replace("", 0)
finalListPlayer["Subs On"] = finalListPlayer["Subs On"].replace("-", 0)
finalListPlayer["Subs On"] = finalListPlayer["Subs On"].replace("", 0)
finalListPlayer["Min"] = finalListPlayer["Min"].replace("-", 0)
finalListPlayer["Min"] = finalListPlayer["Min"].replace("", 0)
finalListPlayer["St"] = finalListPlayer["St"].replace("-", 0)
finalListPlayer["St"] = finalListPlayer["St"].replace("", 0)
finalListPlayer["Position"] = finalListPlayer["Position"].replace("", "-")
finalListPlayer["Age"] = finalListPlayer["Age"].replace("", "-")
finalListPlayer["Height"] = finalListPlayer["Height"].replace("", "-")
finalListPlayer["Foot"] = finalListPlayer["Foot"].replace("", "-")
finalListPlayer["Nation"] = finalListPlayer["Nation"].replace("", "-")

In [29]:
finalListPlayer["MP"] = finalListPlayer["MP"].astype(int)
finalListPlayer["Subs On"] = finalListPlayer["Subs On"].astype(int)

finalListPlayer["St"] = finalListPlayer["MP"].sub(finalListPlayer["Subs On"], fill_value=0)

In [ ]:
uniquePositions = finalListPlayer["Position"].unique()
uniquePositions

In [31]:
finalListPlayer["Position"] = finalListPlayer["Position"].replace("Goalkeeper", "GK")
finalListPlayer["Position"] = finalListPlayer["Position"].replace(
    ["Centre-Back", "Left-Back", "Right-Back"], "DF"
)
finalListPlayer["Position"] = finalListPlayer["Position"].replace(
    [
        "Defensive Midfield",
        "Central Midfield",
        "Right Midfield",
        "Left Midfield",
        "Attacking Midfield",
    ],
    "MF",
)
finalListPlayer["Position"] = finalListPlayer["Position"].replace(
    ["Centre-Forward", "Left Winger", "Right Winger", "Second Striker"], "FW"
)

In [32]:
finalListPlayer["Mini Team Crest"] = ""
finalListPlayer["Big Team Crest"] = ""
finalListPlayer["League"] = ""
finalListPlayer["Team Link"] = ""
finalListPlayer["Team Link 2"] = ""

for p in range(len(finalListPlayer["Team"])):
    for r in range(len(df3["Team"])):
        if finalListPlayer.loc[p, "Team"] == df3.loc[r, "Team"]:
            finalListPlayer.loc[p, "Mini Team Crest"] = df3.loc[r, "Mini Team Crest"]
            finalListPlayer.loc[p, "Big Team Crest"] = df3.loc[r, "Big Team Crest"]
            finalListPlayer.loc[p, "League"] = df3.loc[r, "League"]
            finalListPlayer.loc[p, "Team Link"] = df3.loc[r, "Team Link"]
            finalListPlayer.loc[p, "Team Link 2"] = df3.loc[r, "Team Link 2"]

In [33]:
finalListPlayerMin = finalListPlayer[finalListPlayer["Min"] == 0]
finalListPlayer.drop(
    finalListPlayer[finalListPlayer["Min"] == 0].index,
    inplace=True,
)

finalListPlayer["Min"] = finalListPlayer["Min"].str.replace(".", "")
finalListPlayer["Min"] = finalListPlayer["Min"].str[:-1]
finalListPlayer["Min"] = finalListPlayer["Min"].astype(int)

In [34]:
finalListPlayer = pd.concat([finalListPlayer, finalListPlayerMin], axis=0)

In [35]:
finalListPlayer = finalListPlayer.sort_values(by="Team")
finalListPlayer = finalListPlayer.reset_index(drop=True)

In [48]:
finalListPlayer.insert(10, "St", finalListPlayer.pop("St"))
finalListPlayer.insert(7, "League", finalListPlayer.pop("League"))

In [ ]:
finalListPlayer

In [50]:
df3.insert(10, "Pts/MP", df3.pop("Pts/MP"))
df3.insert(12, "Team Link", df3.pop("Team Link"))
df3.insert(13, "Mini Team Crest", df3.pop("Mini Team Crest"))
df3.insert(13, "Big Team Crest", df3.pop("Big Team Crest"))

df3 = df3.sort_values(by="Pts/MP", ascending=False)
df3 = df3.reset_index(drop=True)

In [ ]:
df3